# CLEAN DATA Notebook

- Google [Docs](https://docs.google.com/document/d/1pvpaBkCJLoyjcH9gHBvy7lUrq6cec-lAayX1MiGizMs/edit)

In [227]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Clean the data

In [228]:
df = pd.read_csv('data/unclean_data.csv').iloc[:,1:]

df['Date'] = pd.to_datetime(df['Date'])
# rename columns
df.rename(columns={'\xa0':'Home','eFG%.1':'D-eFG%','TOV%.1':'D-TOV%','DRB%':'D-DRB%',
                   'FT/FGA.1':'D-FT/FGA','W/L':'Win','Tm':'TmPts','Opp.1':'OppPts'}, inplace = True)
# Drop blank columns
df.drop(df.columns[[33,38]], axis=1, inplace=True)
# Drop missing values and address Home column
df['Home'] =  df['Home'].fillna(1)
df.dropna(axis=0, inplace=True)
df.replace(to_replace ='@', value = 0, inplace =True)
# Address Win/Loss Column
df.replace(to_replace ={'Win': {'W': 1, 'L': 0}}, inplace =True)
# # Check if Wins=Losses or preview df
# df['Win'].value_counts()
# df.head()

### Aggregate Monthly Stats for each game

Use Date to get Month and year to group by. Because October (10) and April (4) have a few games, the monthly stats for these will be joined with November (11) and March (3), respectively.

In [229]:
df.set_index('Date', inplace=True)

In [230]:
df['Year'] = df.index.year
df['Month'] = df.index.month
# df['Day'] = df.index.day
df['Month_to_group'] = df['Month'].replace(to_replace ={10: 11, 4: 3})
df['Yr_M_Tm'] = df[['Year', 'Month_to_group','Team']].apply(tuple, axis=1)
# df.head(5)

,G,Home,Opp,Win,TmPts,OppPts,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,D-eFG%,D-TOV%,D-DRB%,D-FT/FGA,Team,Season,Year,Month,Month_to_group,Yr_M_Tm
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-10-28,1.0,1,IND,1,120.0,109.0,46.0,87.0,0.529,7.0,12.0,0.583,21.0,29.0,0.724,8.0,35.0,30.0,16.0,6.0,10.0,25.0,119.0,108.1,100.8,0.333,0.138,0.601,46.7,65.2,15.9,10.9,0.569,9.1,19.0,0.241,0.603,22.9,81.8,0.288,ATL,2010,2009,10,11,"(2009, 11, ATL)"
2009-10-30,2.0,1,WAS,1,100.0,89.0,32.0,78.0,0.410,5.0,20.0,0.250,31.0,34.0,0.912,11.0,47.0,18.0,5.0,7.0,12.0,24.0,108.5,96.6,92.1,0.436,0.256,0.538,54.0,56.3,5.4,10.6,0.442,11.4,25.6,0.397,0.475,13.6,81.8,0.177,ATL,2010,2009,10,11,"(2009, 11, ATL)"
2009-11-01,3.0,0,LAL,0,110.0,118.0,40.0,89.0,0.449,7.0,23.0,0.304,23.0,26.0,0.885,14.0,40.0,21.0,8.0,3.0,19.0,21.0,109.4,117.4,100.5,0.292,0.258,0.548,48.8,52.5,8.0,4.2,0.489,15.9,31.1,0.258,0.557,13.3,70.3,0.241,ATL,2010,2009,11,11,"(2009, 11, ATL)"
2009-11-03,4.0,0,POR,1,97.0,91.0,37.0,80.0,0.463,5.0,15.0,0.333,18.0,22.0,0.818,11.0,46.0,21.0,4.0,7.0,11.0,15.0,111.6,104.7,86.9,0.275,0.188,0.541,56.1,56.8,4.6,10.9,0.494,10.9,29.7,0.225,0.459,8.0,77.8,0.153,ATL,2010,2009,11,11,"(2009, 11, ATL)"
2009-11-04,5.0,0,SAC,1,113.0,105.0,46.0,85.0,0.541,6.0,16.0,0.375,15.0,19.0,0.789,12.0,47.0,17.0,2.0,10.0,16.0,20.0,120.6,112.1,93.7,0.224,0.188,0.605,52.2,37.0,2.1,12.3,0.576,14.6,33.3,0.176,0.443,6.1,64.8,0.208,ATL,2010,2009,11,11,"(2009, 11, ATL)"


In [231]:
df.reset_index(inplace=True)

Create a monthly stats data frame for each team. Group by year, month and team to get the stat averages. Then, make year, month, and yr/mth/team (tuple) columns to merge to the game logs.

In [232]:
month_stats_df = df.groupby(['Year','Month_to_group','Team']).agg(['mean'])
month_stats_df['Month'] = month_stats_df['Month'].round().astype('int')
month_stats_df['Year'] = [idx[0] for idx in month_stats_df.index]
month_stats_df['Team'] = [idx[2] for idx in month_stats_df.index]
month_stats_df['Yr_M_Tm'] = month_stats_df[['Year', 'Month', 'Team']].apply(tuple, axis=1)
# month_stats_df.head()

G      Home       Win       TmPts     OppPts  \
                         mean      mean      mean        mean       mean   
Year Month_to_group Team                                                   
2009 11             ATL   9.0  0.470588  0.705882  102.411765  97.470588   
                    BOS   9.0  0.588235  0.764706   99.882353  91.294118   
                    CHA   8.5  0.500000  0.437500   88.187500  87.937500   
                    CHI   8.0  0.333333  0.400000   91.400000  97.600000   
                    CLE   9.0  0.470588  0.705882   99.058824  94.588235   

                                 FG        FGA       FG%        3P        3PA  \
                               mean       mean      mean      mean       mean   
Year Month_to_group Team                                                        
2009 11             ATL   39.294118  83.941176  0.469235  6.176471  18.058824   
                    BOS   38.117647  77.470588  0.493765  6.588235  19.294118   
                    CHA   32.812500  76.625000  0.429750  4.625000  16.125000   
                    CHI   35.600000  82.266667  0.433333  3.533333  11.600000   
                    CLE   36.529412  76.470588  0.480059  8.294118  18.941176   

                               3P%         FT        FTA       FT%        ORB  \
                              mean       mean       mean      mean       mean   
Year Month_to_group Team                                                        
2009 11             ATL   0.348882  17.647059  22.647059  0.766588  11.941176   
                    BOS   0.326941  17.058824  22.823529  0.742588   8.823529   
                    CHA   0.271875  17.937500  25.625000  0.697687  11.000000   
                    CHI   0.314600  16.666667  22.466667  0.742933  12.666667   
                    CLE   0.456882  17.705882  24.411765  0.723059   8.529412   

                                TRB        AST       STL       BLK        TOV  \
                               mean       mean      mean      mean       mean   
Year Month_to_group Team                                                        
2009 11             ATL   42.470588  20.352941  6.882353  6.529412  12.235294   
                    BOS   38.235294  24.705882  8.941176  4.823529  14.000000   
                    CHA   42.000000  18.625000  8.125000  4.625000  14.625000   
                    CHI   43.000000  19.466667  7.266667  5.666667  13.800000   
                    CLE   40.000000  20.764706  6.882353  5.411765  13.647059   

                                 PF        ORtg        DRtg       Pace  \
                               mean        mean        mean       mean   
Year Month_to_group Team                                                 
2009 11             ATL   20.411765  111.688235  106.541176  90.976471   
                    BOS   21.294118  109.717647  100.076471  90.464706   
                    CHA   20.125000   98.818750   99.225000  87.812500   
                    CHI   19.333333  100.053333  106.940000  91.460000   
                    CLE   21.647059  109.558824  104.452941  90.535294   

                               FTr      3PAr       TS%       TRB%       AST%  \
                              mean      mean      mean       mean       mean   
Year Month_to_group Team                                                       
2009 11             ATL   0.270059  0.215824  0.545824  50.576471  51.429412   
                    BOS   0.295647  0.249706  0.573176  49.905882  64.776471   
                    CHA   0.342938  0.210938  0.501687  51.168750  56.587500   
                    CHI   0.276200  0.140867  0.496533  49.273333  54.593333   
                    CLE   0.320529  0.247471  0.570647  50.358824  56.823529   

                              STL%      BLK%      eFG%       TOV%       ORB%  \
                              mean      mean      mean       mean       mean   
Year Month_to_group Team                                                      

Dataframe with game logs (no stats for each game)

In [233]:
df_game_logs = pd.concat([df.iloc[:,:5], df.iloc[:,-5:]], axis=1)
# df_game_logs.head()

,Date,G,Home,Opp,Win,Season,Year,Month,Month_to_group,Yr_M_Tm
0,2009-10-28,1.0,1,IND,1,2010,2009,10,11,"(2009, 11, ATL)"
1,2009-10-30,2.0,1,WAS,1,2010,2009,10,11,"(2009, 11, ATL)"
2,2009-11-01,3.0,0,LAL,0,2010,2009,11,11,"(2009, 11, ATL)"
3,2009-11-03,4.0,0,POR,1,2010,2009,11,11,"(2009, 11, ATL)"
4,2009-11-04,5.0,0,SAC,1,2010,2009,11,11,"(2009, 11, ATL)"
5,2009-11-06,6.0,0,CHA,0,2010,2009,11,11,"(2009, 11, ATL)"
6,2009-11-07,7.0,1,DEN,1,2010,2009,11,11,"(2009, 11, ATL)"
7,2009-11-11,8.0,0,NYK,1,2010,2009,11,11,"(2009, 11, ATL)"
8,2009-11-13,9.0,0,BOS,1,2010,2009,11,11,"(2009, 11, ATL)"
9,2009-11-14,10.0,1,NOH,1,2010,2009,11,11,"(2009, 11, ATL)"


In [234]:
gmlgs_df = df_game_logs.merge(month_stats_df.iloc[:,3:], on='Yr_M_Tm')
# gmlgs_df.head()

/Users/martinreyes/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,Date,G,Home,Opp,Win,Season,Year,Month,Month_to_group,Yr_M_Tm,"(TmPts, mean)","(OppPts, mean)","(FG, mean)","(FGA, mean)","(FG%, mean)","(3P, mean)","(3PA, mean)","(3P%, mean)","(FT, mean)","(FTA, mean)","(FT%, mean)","(ORB, mean)","(TRB, mean)","(AST, mean)","(STL, mean)","(BLK, mean)","(TOV, mean)","(PF, mean)","(ORtg, mean)","(DRtg, mean)","(Pace, mean)","(FTr, mean)","(3PAr, mean)","(TS%, mean)","(TRB%, mean)","(AST%, mean)","(STL%, mean)","(BLK%, mean)","(eFG%, mean)","(TOV%, mean)","(ORB%, mean)","(FT/FGA, mean)","(D-eFG%, mean)","(D-TOV%, mean)","(D-DRB%, mean)","(D-FT/FGA, mean)","(Season, mean)","(Month, mean)","(Year, )","(Team, )"
0,2009-10-28,1.0,1,IND,1,2010,2009,10,11,"(2009, 11, ATL)",102.411765,97.470588,39.294118,83.941176,0.469235,6.176471,18.058824,0.348882,17.647059,22.647059,0.766588,11.941176,42.470588,20.352941,6.882353,6.529412,12.235294,20.411765,111.688235,106.541176,90.976471,0.270059,0.215824,0.545824,50.576471,51.429412,7.476471,9.888235,0.506294,11.535294,28.417647,0.210588,0.493706,12.923529,72.876471,0.188,2010,11,2009,ATL
1,2009-10-30,2.0,1,WAS,1,2010,2009,10,11,"(2009, 11, ATL)",102.411765,97.470588,39.294118,83.941176,0.469235,6.176471,18.058824,0.348882,17.647059,22.647059,0.766588,11.941176,42.470588,20.352941,6.882353,6.529412,12.235294,20.411765,111.688235,106.541176,90.976471,0.270059,0.215824,0.545824,50.576471,51.429412,7.476471,9.888235,0.506294,11.535294,28.417647,0.210588,0.493706,12.923529,72.876471,0.188,2010,11,2009,ATL
2,2009-11-01,3.0,0,LAL,0,2010,2009,11,11,"(2009, 11, ATL)",102.411765,97.470588,39.294118,83.941176,0.469235,6.176471,18.058824,0.348882,17.647059,22.647059,0.766588,11.941176,42.470588,20.352941,6.882353,6.529412,12.235294,20.411765,111.688235,106.541176,90.976471,0.270059,0.215824,0.545824,50.576471,51.429412,7.476471,9.888235,0.506294,11.535294,28.417647,0.210588,0.493706,12.923529,72.876471,0.188,2010,11,2009,ATL
3,2009-11-03,4.0,0,POR,1,2010,2009,11,11,"(2009, 11, ATL)",102.411765,97.470588,39.294118,83.941176,0.469235,6.176471,18.058824,0.348882,17.647059,22.647059,0.766588,11.941176,42.470588,20.352941,6.882353,6.529412,12.235294,20.411765,111.688235,106.541176,90.976471,0.270059,0.215824,0.545824,50.576471,51.429412,7.476471,9.888235,0.506294,11.535294,28.417647,0.210588,0.493706,12.923529,72.876471,0.188,2010,11,2009,ATL
4,2009-11-04,5.0,0,SAC,1,2010,2009,11,11,"(2009, 11, ATL)",102.411765,97.470588,39.294118,83.941176,0.469235,6.176471,18.058824,0.348882,17.647059,22.647059,0.766588,11.941176,42.470588,20.352941,6.882353,6.529412,12.235294,20.411765,111.688235,106.541176,90.976471,0.270059,0.215824,0.545824,50.576471,51.429412,7.476471,9.888235,0.506294,11.535294,28.417647,0.210588,0.493706,12.923529,72.876471,0.188,2010,11,2009,ATL


In [235]:
gmlgs_df.drop(gmlgs_df.columns[[8,9,46,47,48]], axis=1, inplace = True)

In [236]:
gmlgs_df.columns = list(gmlgs_df.columns[:8]) + [f'{col[0]}_per_G' for col in gmlgs_df.columns[8:44]] + ['Team']
# gmlgs_df.columns

Index(['Date', 'G', 'Home', 'Opp', 'Win', 'Season', 'Year', 'Month',
       'TmPts_per_G', 'OppPts_per_G', 'FG_per_G', 'FGA_per_G', 'FG%_per_G',
       '3P_per_G', '3PA_per_G', '3P%_per_G', 'FT_per_G', 'FTA_per_G',
       'FT%_per_G', 'ORB_per_G', 'TRB_per_G', 'AST_per_G', 'STL_per_G',
       'BLK_per_G', 'TOV_per_G', 'PF_per_G', 'ORtg_per_G', 'DRtg_per_G',
       'Pace_per_G', 'FTr_per_G', '3PAr_per_G', 'TS%_per_G', 'TRB%_per_G',
       'AST%_per_G', 'STL%_per_G', 'BLK%_per_G', 'eFG%_per_G', 'TOV%_per_G',
       'ORB%_per_G', 'FT/FGA_per_G', 'D-eFG%_per_G', 'D-TOV%_per_G',
       'D-DRB%_per_G', 'D-FT/FGA_per_G', 'Team'],
      dtype='object')

In [237]:
cols = gmlgs_df.columns.tolist()
cols = cols[:3] + cols[-1] + cols[3:44]
gmlgs_df = gmlgs_df[cols] 

In [239]:
gmlgs_df.to_csv('data/clean_data.csv')